In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import json

# 1. データベースの準備 (SQLite)
dbname = 'google_repos.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# テーブルが存在しなければ作成 (id, name, language, stars)
# ※idは自動採番、それ以外はテキストや整数
cur.execute('''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        language TEXT,
        stars INTEGER
    )
''')
conn.commit()

# 2. スクレイピングの実行
# GitHubのAPIを使用（HTMLスクレイピングよりも信頼性が高い）
url = "https://api.github.com/orgs/google/repos"
params = {
    'per_page': 100,  # 1ページあたりの取得件数
    'page': 1
}

print("GitHubのGoogleリポジトリ情報を取得中...")

try:
    response = requests.get(url, params=params)
    
    # APIリクエストが成功したか確認
    if response.status_code == 200:
        repos = response.json()
        
        print(f"{len(repos)} 件のリポジトリが見つかりました。処理を開始します...")
        
        for repo in repos:
            try:
                # リポジトリ情報を取得（JSONから直接）
                repo_name = repo.get('name', 'Unknown')
                language = repo.get('language', 'No Language')
                stars = repo.get('stargazers_count', 0)
                
                # 取得データの確認表示（デバッグ用）
                print(f"取得: {repo_name} | {language} | {stars} stars")
                
                # データベースへ保存
                cur.execute('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)', 
                            (repo_name, language, stars))
                
                # 重要: サーバーへの負荷軽減のため待機
                time.sleep(0.1)
                
            except Exception as e:
                print(f"エラー発生: {e}")
                continue
        
        # 変更を確定
        conn.commit()
    else:
        print(f"APIリクエスト失敗: ステータスコード {response.status_code}")
        
except Exception as e:
    print(f"リクエストエラー: {e}")

# 3. 保存したデータをSELECT文で表示する (課題の要件)
print("\n--- データベース保存結果 ---")
cur.execute('SELECT * FROM repositories ORDER BY stars DESC')
rows = cur.fetchall()

print(f"合計 {len(rows)} 件のリポジトリを保存しました。\n")

for row in rows:
    print(f"ID: {row[0]}, リポジトリ名: {row[1]}, 言語: {row[2]}, スター数: {row[3]}")

# 4. コネクションを閉じる
conn.close()


GitHubのGoogleリポジトリ情報を取得中...
100 件のリポジトリが見つかりました。処理を開始します...
取得: truth | Java | 2778 stars
取得: ruby-openid-apps-discovery | Ruby | 38 stars
100 件のリポジトリが見つかりました。処理を開始します...
取得: truth | Java | 2778 stars
取得: ruby-openid-apps-discovery | Ruby | 38 stars
取得: autoparse | Ruby | 146 stars
取得: anvil-build | Python | 58 stars
取得: autoparse | Ruby | 146 stars
取得: anvil-build | Python | 58 stars
取得: googletv-android-samples | Java | 165 stars
取得: ChannelPlate | JavaScript | 23 stars
取得: googletv-android-samples | Java | 165 stars
取得: ChannelPlate | JavaScript | 23 stars
取得: GL-Shader-Validator | Python | 40 stars
取得: qpp | JavaScript | 43 stars
取得: GL-Shader-Validator | Python | 40 stars
取得: qpp | JavaScript | 43 stars
取得: CSP-Validator | Python | 27 stars
取得: embed-dart-vm | C++ | 99 stars
取得: CSP-Validator | Python | 27 stars
取得: embed-dart-vm | C++ | 99 stars
取得: module-server | JavaScript | 565 stars
取得: cxx-std-draft | TeX | 53 stars
取得: module-server | JavaScript | 565 stars
取得: cxx-std-dra